In [1]:
! pip install flask-ngrok
! pip install Flask-RESTful
! pip install torch==1.5.0
! pip install transformers==2.8.0

In [2]:
! wget "https://bitbucket.org/aryanjadon/transformers_api/src/master/transformers_api/config.json"

--2021-11-29 22:31:06--  https://bitbucket.org/aryanjadon/transformers_api/src/master/transformers_api/config.json
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50268 (49K) [text/html]
Saving to: ‘config.json.1’

config.json.1       100%[===================>]  49.09K  --.-KB/s    in 0.02s   

2021-11-29 22:31:07 (2.35 MB/s) - ‘config.json.1’ saved [50268/50268]



In [3]:
! pip install gdown

In [4]:
import gdown
from pathlib import Path

Path("assets").mkdir(exist_ok=True)

gdown.download(
    "https://drive.google.com/uc?id=1V8itWtowCYnb2Bc9KlK9SxGff9WwmogA",
    "assets/model_state_dict.bin", quiet=False
)

Downloading...
From: https://drive.google.com/uc?id=1V8itWtowCYnb2Bc9KlK9SxGff9WwmogA
To: /content/assets/model_state_dict.bin
100%|██████████| 433M/433M [00:02<00:00, 180MB/s]


'assets/model_state_dict.bin'

In [12]:
import json

from torch import nn
from transformers import BertModel

config = {
    "BERT_MODEL": "bert-base-cased",
    "PRE_TRAINED_MODEL": "assets/model_state_dict.bin",
    "CLASS_NAMES": [
        "negative",
        "neutral",
        "positive"
    ],
    "MAX_SEQUENCE_LEN": 150
}


class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(config["BERT_MODEL"])
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)


In [13]:
import json

import torch
import torch.nn.functional as F
from transformers import BertTokenizer



class Model:
    def __init__(self):

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.tokenizer = BertTokenizer.from_pretrained(config["BERT_MODEL"])

        classifier = SentimentClassifier(len(config["CLASS_NAMES"]))
        classifier.load_state_dict(
            torch.load(config["PRE_TRAINED_MODEL"], map_location=self.device)
        )
        classifier = classifier.eval()
        self.classifier = classifier.to(self.device)

    def predict(self, text):
        encoded_text = self.tokenizer.encode_plus(
            text,
            max_length=config["MAX_SEQUENCE_LEN"],
            add_special_tokens=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        input_ids = encoded_text["input_ids"].to(self.device)
        attention_mask = encoded_text["attention_mask"].to(self.device)

        with torch.no_grad():
            probabilities = F.softmax(self.classifier(input_ids, attention_mask), dim=1)
        confidence, predicted_class = torch.max(probabilities, dim=1)
        predicted_class = predicted_class.cpu().item()
        probabilities = probabilities.flatten().cpu().numpy().tolist()
        return (
            config["CLASS_NAMES"][predicted_class],
            confidence,
            dict(zip(config["CLASS_NAMES"], probabilities)),
        )


model = Model()


def get_model():
    return model


In [ ]:
from flask import Flask, request, jsonify
import tempfile
import os
import subprocess
from werkzeug.utils import secure_filename
import json
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'server_files/'

run_with_ngrok(app)

@app.route('/api/analyze', methods=['GET', 'POST'])
def get_score():
    if request.method == 'POST':
      given_text = request.form.get('analyze_text')

      model = Model()
      sentiment, confidence, probabilities = model.predict(given_text)
      print(sentiment,confidence,probabilities)

      prediction = {
                    "sentiment": sentiment,
                    "confidence": str(confidence),
                    "probabilities": probabilities
                   }
      
      return jsonify(prediction)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6f70-34-68-164-131.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Nov/2021 22:42:02] "POST /api/analyze HTTP/1.1" 200 -


negative tensor([0.8686], device='cuda:0') {'negative': 0.8685904145240784, 'neutral': 0.13022355735301971, 'positive': 0.0011859433725476265}


...

positive tensor([0.9999], device='cuda:0') {'negative': 3.0012617571628653e-05, 'neutral': 3.7551137211266905e-05, 'positive': 0.9999324083328247}


127.0.0.1 - - [29/Nov/2021 22:42:36] "POST /api/analyze HTTP/1.1" 200 -


positive tensor([0.9999], device='cuda:0') {'negative': 6.682259845547378e-05, 'neutral': 3.905816993210465e-05, 'positive': 0.9998940229415894}


127.0.0.1 - - [29/Nov/2021 22:42:52] "POST /api/analyze HTTP/1.1" 200 -


positive tensor([0.9996], device='cuda:0') {'negative': 8.005231211427599e-05, 'neutral': 0.00027683988446369767, 'positive': 0.999643087387085}
